In [49]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [50]:
titanic = sns.load_dataset('titanic')
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [51]:
titanic = df[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'who', 'deck']]
titanic.tail()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,deck
886,0,2,1,27.0,0,0,13.00,2,man,NaN
887,1,1,0,19.0,0,0,30.00,2,woman,B
888,0,3,0,32.8,1,2,23.45,2,woman,NaN
889,1,1,1,26.0,0,0,30.00,0,man,C
890,0,3,1,32.0,0,0,7.75,1,man,NaN


(0, 0, 0)

In [55]:
titanic.age.fillna(titanic[titanic.who.isin(['man', 'woman'])].age.mean().round(1), inplace=True)
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,deck
0,0,3,1,22.0,1,0,7.2500,2,man,NaN
1,1,1,0,38.0,1,0,71.2833,0,woman,C
2,1,3,0,26.0,0,0,7.9250,2,woman,NaN
3,1,1,0,35.0,1,0,53.1000,2,woman,C
4,0,3,1,35.0,0,0,8.0500,2,man,NaN
...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,2,man,NaN
887,1,1,0,19.0,0,0,30.0000,2,woman,B
888,0,3,0,32.8,1,2,23.4500,2,woman,NaN
889,1,1,1,26.0,0,0,30.0000,0,man,C


In [59]:
titanic.embarked.fillna('S', inplace=True)

In [62]:
titanic.drop(columns=['deck', 'who'], inplace=True)

In [63]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [64]:
titanic.sex = le.fit_transform(titanic.sex)
titanic.embarked = le.fit_transform(titanic.embarked)
titanic.tail(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
888,0,3,0,32.8,1,2,23.45,2
889,1,1,1,26.0,0,0,30.00,0
890,0,3,1,32.0,0,0,7.75,1


In [66]:
X = titanic.iloc[:, :-1].values
y = titanic.iloc[:, 0].values

In [67]:
from sklearn.preprocessing import StandardScaler
X_std =  StandardScaler().fit_transform(X)

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    titanic, y, stratify=y, test_size=0.2, random_state=2023
)

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [72]:
lrc = LogisticRegression(random_state=2023)
svc = SVC(random_state=2023)
knn = KNeighborsClassifier(n_neighbors=5)

In [73]:
from sklearn.ensemble import VotingClassifier
voc = VotingClassifier(
    estimators=[('LRC', lrc), ('SVC', svc),('KNN', knn)],
    voting='hard'
)

In [74]:
voc.fit(X_train, y_train)
voc.score(X_test, y_test)

0.8659217877094972

In [75]:
lrc.fit(X_train, y_train)
svc.fit(X_train, y_train)
knn.fit(X_train, y_train)
lrc.score(X_test, y_test), svc.score(X_test, y_test), knn.score(X_test, y_test)

(1.0, 0.6703910614525139, 0.7932960893854749)

In [76]:
svc2 = SVC(probability=True, random_state=2023)
svc2.fit(X_train, y_train)
svc2.predict_proba(X_test[:3])

array([[0.72510332, 0.27489668],
       [0.19806864, 0.80193136],
       [0.72487771, 0.27512229]])

In [77]:
voc2 = VotingClassifier(
    estimators=[('LRC', lrc), ('SVC', svc2),('KNN', knn)],
    voting='soft'
)
voc2.fit(X_train, y_train)
voc2.score(X_test, y_test)

0.9497206703910615

In [78]:
lrc.C, svc.C

(1.0, 1.0)

In [79]:
from sklearn.model_selection import GridSearchCV
params = {
    'LRC__C' : [5, 10, 20],
    'SVC__C' : [0.5, 1 ,3]
}
grid_voc2 = GridSearchCV(voc2, params, scoring='accuracy', cv=5)
grid_voc2.fit(X_train, y_train)
grid_voc2.best_params_

{'LRC__C': 20, 'SVC__C': 3}

In [80]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2023)
rfc.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2023,
 'verbose': 0,
 'warm_start': False}

In [81]:
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

1.0

In [82]:
rfc.predict_proba(X_test[:5])

array([[0.98, 0.02],
       [0.14, 0.86],
       [1.  , 0.  ],
       [1.  , 0.  ],
       [1.  , 0.  ]])

In [83]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
dtc.predict_proba(X_test[:5])

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])